In [1]:
import os
import time
import re
import chromedriver_autoinstaller
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [2]:
# 크롬 버젼 확인 후 새로운 드라이버 설치해서 실행
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
chrome_options.add_argument("--headless") # 리눅스 GUI 없는 디스플레이 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

try: # 크롬 드라이버
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)

# WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
# driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')
driver.implicitly_wait(10) # 10초정도 멈추기

C:\Users\user\AppData\Local\Temp\ipykernel_58168\3550613424.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)
C:\Users\user\AppData\Local\Temp\ipykernel_58168\3550613424.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)


In [3]:
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']

In [6]:
cnt = 0
for index, gu_name in tqdm(enumerate(gu_list)):
    cnt += 1
    fileName = f'{gu_name}_{cnt}.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("카페명" + "|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "대표사진주소" + "\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito") # 시크릿 모드
    chrome_options.add_argument("--headless") # 리눅스 GUI 없는 디스플레이 모드
    chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
    chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
    chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    try: # 크롬 드라이버
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)   
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)

    # WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
    # driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')
    driver.implicitly_wait(10) # 10초정도 멈추기
    
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 카페')  # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element(By.ID, "info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element(By.ID, "info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)
        # place_lists = driver.find_element(By.CSS_SELECTOR, ('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element(By.XPATH, xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(3)
            place_lists = driver.find_elements(By.CSS_SELECTOR, '#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # place_address = store_info.select('.info_item > .addr > p')
                # place_hour = store_info.select('.info_item > .openhour > p > a')
                # place_tel = store_info.select('.info_item > .contact > span')
                 # print("length:", len(place_name))
                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text


                # 사진url 수집
                detail = p.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])

                place_photo = ""
                try:
                    photo = driver.find_elements(By.CSS_SELECTOR, 'span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(place_name, place_photo)

                file.write(place_name + "|" + place_address + "|" + place_hour + "|" + place_tel + "|" + place_photo + "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element(By.ID, "info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")

0it [00:00, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_58168\1680641550.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)
C:\Users\user\AppData\Local\Temp\ipykernel_58168\1680641550.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)


어반플랜트 합정 
피오니 홍대점 
프릳츠 도화점 
943킹스크로스 
카페공명 
오츠커피 마포점 
카페 레이어드 연남점 
비파티세리 공덕점 
앤트러사이트 합정점 
포비 합정점 
앤트러사이트 서교점 
테일러커피 서교점 
당인리책발전소 
무신사테라스 
커피리브레 연남점 
1  of  [  1  ] 
톤앤매너 
1984 
땡스오트 연남점 
빈브라더스 합정점 
오랑지 
콜린 
디벙크 
얼스어스 
땡스네이쳐 
커피가게동경 
샌드커피 논탄토 
키쉬미뇽 
테일러커피 연남1호점 
아이엠베이글 공덕점 
브런치가 
2  of  [  1  ] 
작당모의 
딥블루레이크 
이미커피 
버터밀크 
시루케이크 
콩카페 연남점 
빌리프커피로스터스 
W카페 
무대륙 
비트포비아 홍대던전점 
82년생김민경X카페르세 
허밍벨라 
모아새 
딩가케이크 
제비다방 
3  of  [  1  ] 
카페스콘 
더페이머스램 
포멜로빈 공덕점 
씨스루 홍대연남점 
밤부베이커리&브루잉 
연남방앗간 
히어로보드게임카페 홍대1호점 
카페 클로이인패리스 
그레이랩 
카페 지금여기 
앤디스커피 
공상온도 
카페장쌤 
그로토 
서울이스케이프룸 홍대2호점 
4  of  [  1  ] 
노티드 연남 
브루라티오 홍대점 
라헬의부엌 홍대점 
비로소커피 
랑데자뷰 상수점 
광합성카페 
ESC방탈출카페 홍대점 
민트하임 
커피폴리 
스탠스커피 
도쿄빙수 본점 
네시사분 
멜로우 
미스터리룸이스케이프 홍대2호점 
퍼셉션 (Perception) 
5  of  [  1  ] 
8810리스트레토바 
산리오러버스클럽 
달콤한거짓말 
몽마르뜨언덕위 은하수다방 
카페여유 
이리카페 
펠른 
히어로보드게임카페 홍대2호점 
꼬르소산도 
카페105 
저스티나 
르쁘띠푸 홍대본점 
딥커피 
모파상 
커피냅로스터스 연남동 
1  of  [  2  ] 
카페 사운드웨이브 
티크닉 
어니스트팬케이크 
애프터눈커피 
카페로제 
재미난조각가 홍대점 
딩굴딩굴알타미라 
미어캣프랜즈 
17도씨 
미완성식탁 
비포블루밍 
금옥당 서교점 
다다랩 
리플로우 연남점 

1it [10:42, 642.00s/it]

End of Page
Arrow is Disabled
End of Crawl
앤트러사이트 연희점 
더파이홀 
콘하스 연희점 
매뉴팩트커피 연희본점 
독수리다방 
스웨이커피스테이션 
그릭데이 이대점 
클로리스 신촌점 
벙커1 
하이드미플리즈 
롯지190 
로덴드론 
푸어링아웃 
금옥당 연희점 
호밀밭 
1  of  [  1  ] 
미네르바 
올레무스 
와플잇업 이대2호점 
원앤온리커피 
룸익스케이프 화이트점 
프로토콜 
스타벅스 연희DT점 
티앙팡 오후에 홍차 1호점 
에이투지카페 
디폴트밸류 
베어스덴 
레드버튼 신촌점 
스타벅스 신촌점 
투썸플레이스 신촌점 
보틀라운지 
2  of  [  1  ] 
스타벅스 이대R점 
연희커피 
스타벅스 홍제역점 
스타벅스 가재울뉴타운점 
설빙 신촌점 
롤리홀리 
소설원 
시간이머무는홍차가게 
스타벅스 북가좌DT점 
로네펠트 서대문점 
벤치커피스튜디오 
문베어 
맥널티커피팩토리 
스타벅스 연희동점 
스타벅스 신촌명물거리점 
3  of  [  1  ] 
오월의이유 
포티드 
꽃피다이화다방 
고양이다락방 
페이퍼넛츠 
퐁신 
아이덴티티커피랩 
파파 
노아스로스팅 헤리티지점 
커피빈 신촌점 
카페페라 유니크점 
스타벅스 연대동문점 
스타벅스 남가좌사거리점 
스위스그랜드호텔 에이트리움 
카페 코지 
4  of  [  1  ] 
벨라프라하 
커피크레페 본점 
옥루몽 신촌본점 
투썸플레이스 가재울뉴타운점 
스타벅스 신촌역점 
카페에뚜왈 충정로점 
비하인드리메인 
메가MGC커피 신촌점 
방탈출ESC 
휴 
더클루 신촌프리미엄점 
커피스미스 신촌연대점 
내사랑 
바나프레소 신촌점 
멜로우문 
5  of  [  1  ] 
스타벅스 연세종합관점 
보다더카페 
벌툰 신촌직영점 
카페카페 
카시코이 
보연희 
낟더쌔임 
나무카페 
커피빈 연대동문회관점 
37.5 DMC가재울점 
논탄토 신촌점 
스타벅스 연세백양로점 
할리스 신촌점 
탈리 
디저트39 북아현점 
1  of  [  2  ] 
스타벅스 이대ECC점 
발루토 핀치 
까페여름 
스타벅스 

2it [23:54, 730.26s/it]

...

End of Page
Arrow is Disabled
End of Crawl
커피별녹색잔 
카페제하 
더브라운 목동점 
스타벅스 신월ICDT점 
스타벅스 신정DT점 
하니앤손스 
헬로방방 목동점 
퍼블릭맨션 
패닉이스케이프 목동지점 
카페꼬숩 
스타벅스 오목교역점 
아모르버터 
도쿄빙수 오목교역점 
달다랗다 
스타벅스 목동역점 
1  of  [  1  ] 
파티세리소나 2호점 
스타벅스 신월동DT 
카페올리 
스타벅스 목동8단지점 
카페스페이스530 
비엔나커피하우스 목동7단지점 
스타벅스 목동10단지점 
스타벅스 등촌역점 
카페히든 
카페화목토 
스타벅스 목동오목로점 
설빙 목동점 
코나빈스 
커피집 이공 
투썸플레이스 목동하이페리온점 
2  of  [  1  ] 
테디랩 
스타벅스 목동점 
카페아리네 
이디야커피 오목교대학학원점 
스타벅스 목동파라곤점 
퀸즈브라운 목동역점 
라헬의부엌 목동점 
에프스토리 
리모트 
투썸플레이스 신정네거리점 
커피그레이드 
투썸플레이스 남부지방법원점 
스타벅스 목동7단지점 
마스터그레이드커피 
달달애 
3  of  [  1  ] 
제이블리키즈카페 
스타벅스 현대목동점 
러브 바리스타 
할리스 목동점 
커피빈 오목교역사거리점 
투썸플레이스 신정뉴타운점 
프레쉬모먼트카페 
신월커피 
스타벅스 목동행복한점 
카페멍 목동점 
뉴웨이브 커피 
뉴웨이브 커피로스터스 
투썸플레이스 양천구청점 
노유민코페 신월직영점 
빌리엔젤 목동41타워점 
4  of  [  1  ] 
디저트39 목동점 
투썸플레이스 목동메가박스점 
카페동네 목동점 
비엔나커피하우스 목동역점 
투썸플레이스 염창역점 
투썸플레이스 양천신목로점 
타조메롱 
투썸플레이스 신월IC DI점 
홀리빈 카페 
스타벅스 목동파리공원점 
할리스 신정네거리역점 
스타벅스 목동5단지점 
스타벅스 목동트라팰리스점 
놀숲 목동오목교점 
커피하오 
5  of  [  1  ] 
마녀의만화소굴 신정네거리역점 
놀숲 신정네거리역점 
파스쿠찌 목동하이페리온2점 
메가MGC커피 목동시장점 
커스텀커피 목

24it [5:21:47, 900.16s/it]

End of Page
Arrow is Disabled
End of Crawl
고양이정원 
카페우드진 
고양이똥 
브루클린커피 
헤리스헤이스 
어나더사이드 
브런치팩토리 
광화문살롱 
인라이크 
스타벅스 화곡DT점 
스타벅스 마곡나루 
고양이똥 3호점 
카페움플레이트 
스타벅스 방화DT 
구도커피 
1  of  [  1  ] 
스타벅스 마곡사이언스타워R 
벌툰 마곡발산점 
맥켄지커피 까치산역점 
스타벅스 화곡동점 
스타벅스 발산역 
헤이더그린 
빌리엔젤 마곡점 
비엔나커피하우스 마곡나루역점 
쉐드커피 
카페비쥬 
스타벅스 마곡역 
스타벅스 김포공항DT점 
스타벅스 까치산역점 
고양이후추 
플루이드 
2  of  [  1  ] 
이층창가 (휴업중) 
꿀팟만화보드카페 
스타벅스 마곡나루공원 
빌리엔젤 마곡역점 
153도씨 
OMW애견카페 
투썸플레이스 마곡점 
봉봉키즈 강서점 
카페소금다방 
스타벅스 등촌 
와밀 
카페 우드진 마곡나루점 
포퀸스보드카페 
투썸플레이스 마곡나루역점 
헬로방방 마곡점 
3  of  [  1  ] 
스타벅스 마곡GMG 
스타벅스 양천향교역 
스타벅스 강서구청사거리점 
스타벅스 가양역점 
포노 
카카오독애견카페 목동점 
세라젬웰카페 서울마곡직영점 
스타벅스 염창역점 
벌툰 까치산역점 
커피쿡 발산점 
미스터힐링 마곡발산역점 
폴바셋 강서마곡DT점 
투썸플레이스 강서송정역점 
스타벅스 우장산역 
투썸플레이스 등촌역점 
4  of  [  1  ] 
스타벅스 등촌대로점 
시크릿쥬쥬&또봇 프리미엄키즈카페 발산점 
카페드도서관 
커피빈 마곡마커스빌딩점 
브라운씨 
스타벅스 가양역사거리점 
스타벅스 발산역사거리점 
펠어커피초코 마곡점 
오프레스트 로스터리 
설빙 발산점 
투썸플레이스 까치산역점 
몬스터파크 가양점 
카페희다 마곡점 
스타벅스 내발산점 
삐삐 
5  of  [  1  ] 
캐시아커피하우스 
설빙 서울화곡역점 
밀화당 강서직영점 
투썸플레이스 강서구청점 
커피볶는곤이네집 
체르보 
아프리카커피 
투썸플레이스 마곡역점 
폴바셋 롯데김포공항

25it [5:34:25, 802.63s/it]

End of Page
Arrow is Disabled
End of Crawl
